In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
import time

# Set random seed for reproducibility
torch.manual_seed(42)

# Define transforms and datasets
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.Flowers102(root='./dataset', split='train', transform=train_transform, download=True)
test_dataset = torchvision.datasets.Flowers102(root='./dataset', split='val', transform=test_transform, download=True)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

# Initialize the model
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 102)  # Change the last layer for 102 classes

# Define loss function, optimizer, and scheduler
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # StepLR scheduler

# Train the model
def train():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return total_loss / len(train_dataloader), correct / total

# Test the model
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return correct / total


def main():
    best_acc = 0.0
    epochs_without_improvement = 0
    max_epochs_without_improvement = 10
    epoch = 0
    accuracy_threshold = 0.75
    while True:
        start_time = time.time()
        train_loss, train_acc = train()
        test_acc = test()
        end_time = time.time()
        epoch_duration = end_time - start_time

        print(f"Epoch [{epoch+1}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Time: {epoch_duration:.2f}s")
        scheduler.step()

        if test_acc > best_acc:
            torch.save(model.state_dict(), 'best_model.pth')
            best_acc = test_acc
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Check if accuracy exceeds the threshold, and print Accuracy id >= 75%
        if test_acc >= accuracy_threshold:
            print(f"Accuracy exceeds {round(test_acc * 100)}%! \n")

        if epochs_without_improvement >= max_epochs_without_improvement:
            print("Early stopping due to no improvement in validation accuracy")
            break

        epoch+=1

if __name__ == '__main__':
    main()


Epoch [1], Train Loss: 4.0585, Train Acc: 0.1396, Test Acc: 0.1049, Time: 90.96s
Epoch [2], Train Loss: 2.6347, Train Acc: 0.3958, Test Acc: 0.2873, Time: 84.22s
Epoch [3], Train Loss: 1.9788, Train Acc: 0.5135, Test Acc: 0.4059, Time: 83.75s
Epoch [4], Train Loss: 1.6354, Train Acc: 0.6083, Test Acc: 0.5088, Time: 84.48s
Epoch [5], Train Loss: 1.2681, Train Acc: 0.7021, Test Acc: 0.5843, Time: 84.88s
Epoch [6], Train Loss: 0.9310, Train Acc: 0.7885, Test Acc: 0.7569, Time: 84.53s
Accuracy exceeds 76%! 

Epoch [7], Train Loss: 0.7664, Train Acc: 0.8385, Test Acc: 0.7863, Time: 85.80s
Accuracy exceeds 79%! 

Epoch [8], Train Loss: 0.6542, Train Acc: 0.8802, Test Acc: 0.7951, Time: 87.68s
Accuracy exceeds 80%! 

Epoch [9], Train Loss: 0.6582, Train Acc: 0.8646, Test Acc: 0.8206, Time: 85.21s
Accuracy exceeds 82%! 

Epoch [10], Train Loss: 0.5467, Train Acc: 0.9010, Test Acc: 0.8216, Time: 83.12s
Accuracy exceeds 82%! 

Epoch [11], Train Loss: 0.5726, Train Acc: 0.9031, Test Acc: 0.8255, 